In [0]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.layers.core import Activation,Flatten,Dense,Dropout
from keras import backend as keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from keras.optimizers import Adam,SGD,RMSprop,Adagrad,Adadelta,Adamax,Nadam
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

#C+MP+DO+(2C+MP+DO)+Flatten+Dense(relu)+BN+DO+Dense(Softmax)
#C=CONV+RELU+BN

class tinyVGG:
  @staticmethod
  def build(height,width,depth,classes,opt='Adam',learn_rate=0.001,moment=0,init_mode='uniform',dropout=0.0):
    model = Sequential()
    input_shape = (height,width,depth)
    channel_dim = -1
    if(keras.image_data_format()=='channels_first'):
      input_shape = (depth, height, width)
      channel_dim = 1
   
    model.add(Conv2D(32,(3,3),padding='same',input_shape=input_shape,kernel_initializer=init_mode))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(Dropout(0.25+dropout))
    
    model.add(Conv2D(64,(3,3),padding='same',input_shape=input_shape,kernel_initializer=init_mode))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(Conv2D(64,(3,3),padding='same',input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25+dropout))
    
    model.add(Conv2D(128,(3,3),padding='same',input_shape=input_shape,kernel_initializer=init_mode))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(Conv2D(128,(3,3),padding='same',input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25+dropout))

    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5+dropout))
    model.add(Dense(7))
    model.add(Activation('softmax'))

    if opt=='Adam':
      opt=Adam(lr=learn_rate)
    
    if opt=='SGD':
      opt=SGD(lr=learn_rate,momentum=moment)
    
    if opt=='RMSprop':
      opt=RMSprop(lr=learn_rate)

    if opt=='Adagrad':
      opt=Adagrad(lr=learn_rate)

    if opt=='Adadelta':
      opt=Adadelta(lr=learn_rate)
    
    if opt=='Adamax':
      opt=Adamax(lr=learn_rate)

    if opt=='Nadam':
      opt=Nadam(lr=learn_rate)

    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])  
    return model

	

In [0]:

HP_LR= 1e-3
HP_EPOCHS=50
HP_BS=32
HP_IMAGE_DIM=(96,96,3)
data=[]
classes=[]

data1=pd.read_csv('/content/drive/My Drive/data_NUS/icml_face_data.csv')
y=np.array(data1['0'])
xt=np.array(data1)
xt=np.array(data1.drop('0',axis=1))
xt=xt.reshape(35886,48,48,1)
xt=xt/255.0

In [0]:
labels = np.array(y)
lb=LabelBinarizer()
labels = lb.fit_transform(labels)
xtrain,xtest,ytrain,ytest = train_test_split(xt,labels,test_size=0.2,random_state=42)

In [0]:
'''
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

datagen.fit(xtrain)
'''

'''aug = ImageDataGenerator(rotation_range=0.25, width_shift_range = 0.1,height_shift_range=0.1,shear_range=0.2,
                         zoom_range=0.2,horizontal_flip = True,fill_mode='nearest')
aug.fit(xtrain)'''
#model = tinyVGG.build(height=48,width=48,depth=1,classes=7)
#opt = Adam(lr=HP_LR, decay= HP_LR/HP_EPOCHS)

'''H = model.fit_generator(aug.flow(xtrain,ytrain,batch_size=HP_BS),validation_data=(xtest,ytest),
                        steps_per_epoch=len(xtrain)//HP_BS,epochs=HP_EPOCHS)'''

'''H = model.fit_generator(datagen.flow(xtrain, ytrain, batch_size=32),validation_data=(xtest,ytest),
                    steps_per_epoch=len(xtrain) // 32, epochs=100)

model.save('/content/drive/My Drive/Models/my_model_test2.h5')
'''


"H = model.fit_generator(datagen.flow(xtrain, ytrain, batch_size=32),validation_data=(xtest,ytest),\n                    steps_per_epoch=len(xtrain) // 32, epochs=100)\n\nmodel.save('/content/drive/My Drive/Models/my_model_test2.h5')\n"

In [0]:
''' 
from keras.models import load_model

model = load_model('/content/drive/My Drive/Models/my_model_test1.h5')

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
w=xtest
classes = model.predict_classes(xtest)

#model.summary(xtest)
score =model.evaluate(xtest,ytest)

print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

'''


' \nfrom keras.models import load_model\n\nmodel = load_model(\'/content/drive/My Drive/Models/my_model_test1.h5\')\n\nmodel.compile(loss=\'categorical_crossentropy\',\n              optimizer=\'adam\',\n              metrics=[\'accuracy\'])\nw=xtest\nclasses = model.predict_classes(xtest)\n\n#model.summary(xtest)\nscore =model.evaluate(xtest,ytest)\n\nprint("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))\n\n'

In [0]:
#The code to figrue out the optimal batchsize and epcohs
model=KerasClassifier(build_fn=tinyVGG.build,height=48,width=48,depth=1,classes=7,verbose=1)
batch_sizes=[16,32]
epoch_sizes=[10,50]
params_grid=dict(batch_size=batch_sizes,epochs=epoch_sizes)
grid=GridSearchCV(estimator=model,param_grid=params_grid,n_jobs=-1,cv=3)
grid_result=grid.fit(xtrain,ytrain)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

KeyboardInterrupt: ignored

In [0]:
#Setting the number of epochs and batchsize to be equal to the values found in the last GridSearch

epochs=grid_result.best_params_['epochs']
batch_size=grid_result.best_params_['batch_size']


In [0]:
#Code for maximizing the output wrt loss function
model=KerasClassifier(build_fn=tinyVGG.build,epochs=epochs,batch_size=batch_size,verbose=1)
optimizers=['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
params_grid = dict(optimizer=optimizers)
grid = GridSearchCV(estimator=model, param_grid=params_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(xtrain,ytrain)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

NameError: ignored

In [0]:
#Setting the optimizer to the  optimal value found in the last gridsearch
optimizer=grid_result.best_params_['optimizer']


In [0]:
#Code for optimizing learning rates and momentum
model=KerasClassifier(build_fn=tinyVGG.build,epochs=epochs,batch_size=batch_size,opt=optimizer,verbose=1)
learn_rates=[0.001,0.01,0.1,0.2,0.3]
momentum=[0.0,0.1,0.3,0.5,0.7,0.9]
params_grid=dict(learn_rate=learn_rates,momentum=momentum)
grid=GridSearchCV(estimator=model,param_grid=params_grid,n_jobs=-1,cv=3)
grid_result=grid.fit(xtrain,ytrain)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


In [0]:
#Setting the respective values of learning rate and momentum
learn_rate=grid_result.best_params_['learn_rate']
momentum=grid_result.best_params_['momentum']


In [0]:
#Optimization with respect to weight initialisation
model=KerasClassifier(build_fn=tinyVGG.build,epochs=epochs,batch_size=batch_size,opt=optimizer,learn_rate=learn_rate,moment=momentum,verbose=1)
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(xtrain,ytrain)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


NameError: ignored

In [0]:
#Setting the value of weight initialisation to the best parameter found
init_mode=grid_result.best_params_['init_mode']

In [0]:
#Optimizing Dropout rates to kill overfitting
model=KerasClassifier(build_fn=tinyVGG.build,epochs=epochs,batch_size=batch_size,opt=optimizer,learn_rate=learn_rate,moment=momentum,init_mode=init_mode,verbose=1)
dropout=[0.0,0.1,0.2,0.3,0.4]
param_grid=dict(dropout_rate=dropout)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(xtrain,ytrain)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))



NameError: ignored

In [0]:
#Setting the optimal dropout 

dropout_rate=grid_result.best_params_['dropout_rate']
